In [1]:
import pandas as pd
import numpy as np

In [ ]:
results_df = pd.read_csv('./datasets/2014-15 Championship Results.csv')
results_df

In [ ]:
results_df = results_df[results_df.Score.notnull()]
results_df.drop(['Referee', 'Match Report', 'Notes', 'Attendance'], axis=1, inplace = True)
results_df = results_df[results_df.Round == "Regular Season"]
results_df

In [ ]:
results_df['Home_Goals'] = results_df['Score'].astype(str).str[0]
results_df['Away_Goals'] = results_df['Score'].astype(str).str[-1]
results_df

In [ ]:
results_df.dtypes

In [ ]:
results_df.Wk = results_df.Wk.astype(int)
results_df.Home_Goals = results_df.Home_Goals.astype(int)
results_df.Away_Goals = results_df.Away_Goals.astype(int)
results_df.dtypes

In [ ]:
results_df.drop(['Score'], axis=1, inplace = True)
results_df = results_df[['Round','Wk', 'Day', 'Time', 'Home', 'Home_Goals', 'Away', 'Away_Goals', 'Venue']]
results_df

In [ ]:
def calculateWinner(df):
    if df.Home_Goals > df.Away_Goals:
        return df.Home
    elif df.Home_Goals < df.Away_Goals:
        return df.Away
    elif df.Home_Goals == df.Away_Goals:
        return "Draw"

In [ ]:
results_df['Winner'] = results_df.apply(calculateWinner, axis=1)
results_df

In [ ]:
len(results_df.Winner.unique())

In [ ]:
clubNames = list(results_df.Home.unique())
pointsTable = {}
for club in clubNames:
    pointsTable[club] = [0,0,0,0,0,0,0] #Matches Played, Won, Lost, Drawn, Points, GF, GA
pointsTable

In [ ]:
brentford_stats = []
for wkNo in range(1,47):
    results_wk = results_df[results_df['Wk'] == wkNo]
    for index, row in results_wk.iterrows():
        if row['Winner'] == "Draw":
            current_table_home = pointsTable[row['Home']]
            current_table_away = pointsTable[row['Away']]
            current_table_home[0] += 1
            current_table_home[3] += 1
            current_table_home[4] += 1
            current_table_home[5] += row['Home_Goals']
            current_table_home[6] += row['Away_Goals']
            current_table_away[0] += 1
            current_table_away[3] += 1
            current_table_away[4] += 1
            current_table_away[5] += row['Away_Goals']
            current_table_away[6] += row['Home_Goals']
            pointsTable[row['Home']] = current_table_home
            pointsTable[row['Away']] = current_table_away
        elif row['Winner'] == row['Home']:
            current_table_home = pointsTable[row['Home']]
            current_table_away = pointsTable[row['Away']]
            current_table_home[0] += 1
            current_table_home[1] += 1
            current_table_home[4] += 3
            current_table_home[5] += row['Home_Goals']
            current_table_home[6] += row['Away_Goals']
            current_table_away[0] += 1
            current_table_away[2] += 1
            current_table_away[5] += row['Away_Goals']
            current_table_away[6] += row['Home_Goals']
            pointsTable[row['Home']] = current_table_home
            pointsTable[row['Away']] = current_table_away
        elif row['Winner'] == row['Away']:
            current_table_home = pointsTable[row['Home']]
            current_table_away = pointsTable[row['Away']]
            current_table_home[0] += 1
            current_table_home[2] += 1
            current_table_home[5] += row['Home_Goals']
            current_table_home[6] += row['Away_Goals']
            current_table_away[0] += 1
            current_table_away[1] += 1
            current_table_away[4] += 3
            current_table_away[5] += row['Away_Goals']
            current_table_away[6] += row['Home_Goals']
    pointsTable_sorted = sorted(list(pointsTable.items()), key = lambda x : (x[1][4], x[1][5]-x[1][6], x[1][5]), reverse = True)
    gw_stat = dict()
    gw_stat['Round_Number'] = wkNo
    gw_stat['Position'] = [i for i, tupl in enumerate(pointsTable_sorted) if tupl[0] == 'Brentford'][0] + 1
    gw_stat['Points'] = dict(pointsTable_sorted)['Brentford'][4]
    gw_stat['Goals_Scored'] = dict(pointsTable_sorted)['Brentford'][5]
    brentford_stats.append(gw_stat)
brentford_stats

In [ ]:
brentford_stats_df = pd.DataFrame(brentford_stats)
brentford_stats_df = brentford_stats_df[['Round_Number', 'Position', 'Points', 'Goals_Scored']]
brentford_stats_df['Season'] = "2014-15"
brentford_stats_df.to_csv('BrentfordStats2014-15.csv', sep=',', index=False)

In [ ]:
dict(pointsTable_sorted)['Brentford'][5]

In [ ]:
index = [i for i, tupl in enumerate(pointsTable_sorted) if tupl[0] == 'Brentford'][0]
index

In [32]:
departures = pd.read_csv('./datasets/Brentford Transfer Departures.csv')
departures['Transfer Type'] = None

In [33]:
for index, row in departures.iterrows():
    row['Player'] = row['Player'][:-1]
    row['To'] = row['To'][:-1]
    
    if row['Transfer Fee'][0] == "£":
        row['Transfer Fee'] = row['Transfer Fee'][1:]
    
    if row['Transfer Fee'] == "?" or row['Transfer Fee'] == "-":
        row['Transfer Fee'] = None
    
    if row['Transfer Fee'] is not None and row['Transfer Fee'][-1] == "m":
        row['Transfer Fee'] = row['Transfer Fee'][:-1]
        
    if row['Transfer Fee'] is not None and row['Transfer Fee'].endswith('Th.'):
        row['Transfer Fee'] = "0." + row['Transfer Fee'][:-3]
    
    if row['Transfer Fee'] == "End of loan" or row['Transfer Fee'] == "Free transfer" or row['Transfer Fee'] == "Loan":
        row['Transfer Type'] = row['Transfer Fee']
        row['Transfer Fee'] = None
    else:
        row['Transfer Type'] = "Direct"
departures.to_csv('Brentford Transfer Departures.csv', index=False)

In [23]:
player_stats = pd.read_csv('./datasets/Championship Player Stats.csv')
char = '\\'

In [24]:
for index, row in player_stats.iterrows():
    i = row['Player'].find(char)
    temp = row['Player'][:i]
    row['Player Name'] = temp
    player_stats.set_value(index,'Player',temp)
player_stats.to_csv('Championship Player Stats.csv')

c:\users\pg\appdata\local\programs\python\python37-32\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  """
